In [5]:
import string
from matplotlib.animation import FFMpegWriter
import matplotlib
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!cd D: \GITHUB\GitHub\datasetProcessingForSUMO
!pwd


###################################################################################################
#############主程序
#matplotlib.use("Agg")
print(FFMpegWriter.bin_path())

df = pd.read_csv('.\data\data1.csv', sep=';')
print("#################################################################")
print("df.head()\n", df.head())
#print("df.info()\n",df.info())
#print(df.describe())
veh_lane1 = df[df.vehicle_lane == '-1356_0']
print("#################################################################")
print("veh_lane1\n", veh_lane1)

veh_lane = df.vehicle_lane
print("#################################################################")
#print("veh_lane.unique()\n",veh_lane.unique())
print("#################################################################")

print("#################################################################")
print("显示多车同一车道")
print("#################################################################")


df1 = df.groupby(["vehicle_lane", "timestep_time"]).size()
#print(df1)
col = df1[df1 > 1].reset_index()[["vehicle_lane", "timestep_time"]]
#print(col)
mulVehsOneLane = pd.merge(col, df, on=["vehicle_lane", "timestep_time"])
print(mulVehsOneLane)
#input()

print("#################################################################")
print("显示轨迹")
print("#################################################################")


fg1 = plt.figure(figsize=(8, 5))
metadata = dict(title='france Dataset traces',
                artist='lukeliuli', comment='france Dataset traces')
writer = FFMpegWriter(fps=5, metadata=metadata)


#df = mulVehsOneLane
for i, curLaneID in enumerate(df.vehicle_lane.unique()):  # 枚举每一个车道

    vehInOneLane = df[df.vehicle_lane == curLaneID]
    maxLanePos = max(vehInOneLane.vehicle_pos)
    vehInOneLane = vehInOneLane.sort_values(by='timestep_time', ascending=True)

    redFlagRecord = vehInOneLane.drop(index=vehInOneLane.index)

    #画车道图
    lanePosX = vehInOneLane.vehicle_x.to_numpy()
    lanePosY = vehInOneLane.vehicle_y.to_numpy()
    lanePosXT = lanePosX[:len(lanePosX):10]
    lanePosYT = lanePosY[:len(lanePosY):10]
    #plt.plot( lanePosXT,lanePosYT, '.',color = 'black',linewidth=1,markersize=0.1,label='lane')

    #writer.setup(fg1, str(i)+"-"+"laneID("+str(curLaneID)+").mp4", dpi=600)
    #提取时刻
    timeList = vehInOneLane.timestep_time.unique()
    lowSpeedFlag = 0
    for t in timeList:  # 枚举每个时间
        title = "LaneID:"+str(curLaneID)+";"+"Time:"+str(t)+";"
        print(title)

        colorList = ['r', 'g', 'b', 'y', 'c', 'm', 'k']

        vehsAtTime = vehInOneLane[vehInOneLane.timestep_time == float(t)]

        for index, veh in vehsAtTime.iterrows():  # 每一辆车
            #print(index,veh)
            vehX = veh.vehicle_x
            vehY = veh.vehicle_y
            vehVel = veh.vehicle_speed
            vehTime = veh.timestep_time
            vehID = veh.vehicle_id
            vehicle_pos = veh.vehicle_pos
            #title = title+"\n"+vehID+",Speed:"+str(vehVel)+";"

            randNum = sum([ord(x) for x in str(vehID)])
            #plt.plot( vehX,vehY, 'o',color= colorList[randNum%7],label=vehID)  # 蓝色圆点实线
            #plt.annotate(str(vehVel)+","+str(vehicle_pos)+','+vehID, xy=(vehX,vehY),  xytext=(-50, 10), textcoords='offset points')
            #plt.plot( lanePosXT,lanePosYT, '.',color = 'black',markersize=0.1,label='lane')
            if vehVel < 1 & (abs(vehicle_pos - maxLanePos) < 1):
                lowSpeedFlag = 1
                #plt.annotate("redLight", xy=(vehX,vehY),  xytext=(-50, 20), textcoords='offset points')
                #print("redLight"+vehID)
                redFlagRecord = redFlagRecord.append(veh, ignore_index=True)

        #plt.legend()
        #plt.title(title,fontsize='xx-small')
        #plt.show()
        #writer.grab_frame()
        #plt.ioff()
        #plt.cla()

    ######对每条道路进行分析,当有车处于红灯状态时而且车道上有多个车时，给出红灯状态持续时间
    redVehs = redFlagRecord.vehicle_id.unique()  # 红灯状态的车辆ID
    for i, id in enumerate(redVehs):

        #红灯状态的车辆ID
        # 红灯状态的车辆ID
        redFlagRecordTMP = redFlagRecord[redFlagRecord.vehicle_id == id]
        timeList = redFlagRecordTMP.timestep_time  # 红灯状态持续时间
        indexTmp = (vehInOneLane.timestep_time >= min(timeList)) & (
            vehInOneLane.timestep_time <= max(timeList))
        vehsIDs = vehInOneLane[indexTmp]  # 获得红灯状态持续时间内，车道内的车辆数目

        if len(vehsIDs.vehicle_id.unique()) > 1:
            redLightTime = timeList  # 时间
            vehsAtRedTime = vehsIDs  # 红灯时间内车道状态
            redTimeDataSet = redFlagRecordTMP  # 红灯时间的红灯车状态
            dict = {}

            for i, idTmp in enumerate(vehsAtRedTime.vehicle_id.unique()):
                if idTmp == id:
                    continue
                input()
                index = (vehsAtRedTime.vehicle_id == idTmp) & (
                    maxLanePos - vehsAtRedTime.vehicle_pos) < 100
                vehTMP = vehsAtRedTime[index]
                minSpeed = min(vehTmp.vehicle_speed)
                if minSpeed >= 40/3.6:
                    speedFlag = 4
                if minSpeed < 40/3.6 & minSpeed > 30/3.6:
                    speedFlag = 3
                if minSpeed < 30/3.6 & minSpeed > 20/3.6:
                    speedFlag = 2
                if minSpeed < 20/3.6 & minSpeed > 10/3.6:
                    speedFlag = 1
                if minSpeed < 10/3.6:
                    speedFlag = 0
                dict[idTmp] = speedFlag

            for t in timeList:  # 枚举每个时间
                vehsAtTime = vehInOneLane[vehInOneLane.timestep_time == float(
                    t)]
                vehsAtTime = vehsAtTime.sort_values(
                    by='lane_pos', ascending=True)

                samples = []
                for index, veh in vehsAtTime.iterrows():  # 每一辆车
                    vehX = veh.vehicle_x
                    vehY = veh.vehicle_y
                    vehVel = veh.vehicle_speed
                    vehTime = veh.timestep_time
                    vehID = veh.vehicle_id
                    vehicle_pos = veh.vehicle_pos

                    samples2 = []

                    if index > 0:
                        avg_speed_lane = 60/3.6
                        max_speed_lane = 80/3.6
                        subject = [max(timeList) - t,
                                   vehicle_pos,
                                   speed,
                                   vehicle_pos/speed,
                                   vehicle_pos/avg_speed_lane,
                                   vehicle_pos/max_speed_lane]
                        samples1 = samples.extend([0, 0]*(8-index))
                        samples2 = samples2.append(
                            subject.extend(samples1).extend(dict[vehID]))
                        print(samples2)
                        input()

                    samples.extend([vehicle_pos, speed])


    #writer.finish()


#plt.plot(df.vehicle_x,df.vehicle_y, '.')  # 蓝色圆点实线
#plt.show()


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

LaneID:28925528_0;Time:603.0;
LaneID:28925528_0;Time:604.0;
LaneID:28925528_0;Time:605.0;
LaneID:28925528_0;Time:606.0;
LaneID:28925528_0;Time:611.0;
LaneID:28925528_0;Time:612.0;
LaneID:28925528_0;Time:613.0;
LaneID:28925528_0;Time:614.0;
LaneID:28925528_0;Time:615.0;
LaneID:28925528_0;Time:616.0;
LaneID:28925528_0;Time:617.0;
LaneID:28925528_0;Time:618.0;
LaneID:28925528_0;Time:619.0;
LaneID:28925528_0;Time:620.0;
LaneID:28925528_0;Time:621.0;


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

LaneID:28925528_0;Time:622.0;
LaneID:28925528_0;Time:623.0;
LaneID:28925528_0;Time:624.0;
LaneID:28925528_0;Time:625.0;
LaneID:28925528_0;Time:626.0;
LaneID:28925528_0;Time:627.0;


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

...

LaneID:28925528_0;Time:7052.0;
LaneID:28925528_0;Time:7053.0;
LaneID:28925528_0;Time:7054.0;
LaneID:28925528_0;Time:7055.0;
LaneID:28925528_0;Time:7056.0;
LaneID:28925528_0;Time:7057.0;
LaneID:28925528_0;Time:7058.0;
LaneID:28925528_0;Time:7059.0;
LaneID:28925528_0;Time:7081.0;
LaneID:28925528_0;Time:7082.0;
LaneID:28925528_0;Time:7083.0;
LaneID:28925528_0;Time:7084.0;
LaneID:28925528_0;Time:7085.0;
LaneID:28925528_0;Time:7086.0;
LaneID:28925528_0;Time:7087.0;
LaneID:28925528_0;Time:7110.0;
LaneID:28925528_0;Time:7111.0;
LaneID:28925528_0;Time:7112.0;
LaneID:28925528_0;Time:7113.0;
LaneID:28925528_0;Time:7114.0;
LaneID:28925528_0;Time:7115.0;
LaneID:28925528_0;Time:7116.0;
LaneID:28925528_0;Time:7138.0;
LaneID:28925528_0;Time:7139.0;
LaneID:28925528_0;Time:7140.0;
LaneID:28925528_0;Time:7141.0;
LaneID:28925528_0;Time:7142.0;
LaneID:28925528_0;Time:7143.0;
LaneID:28925528_0;Time:7144.0;
LaneID:28925528_0;Time:7145.0;
LaneID:28925528_0;Time:7146.0;
LaneID:28925528_0;Time:7147.0;
LaneID:2

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

LaneID:28925528_0;Time:7176.0;
LaneID:28925528_0;Time:7177.0;
LaneID:28925528_0;Time:7178.0;
LaneID:28925528_0;Time:7192.0;
LaneID:28925528_0;Time:7193.0;
LaneID:28925528_0;Time:7194.0;
LaneID:28925528_0;Time:7195.0;
LaneID:28925528_0;Time:7196.0;
LaneID:28925528_0;Time:7197.0;
LaneID:28925528_0;Time:7198.0;
LaneID:28925528_0;Time:7199.0;
LaneID:28925528_0;Time:7200.0;
LaneID:28925528_0;Time:7201.0;
LaneID:28925528_0;Time:7202.0;
LaneID:28925528_0;Time:7203.0;
LaneID:28925528_0;Time:7204.0;
LaneID:28925528_0;Time:7205.0;


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  redFlagRecord = redFlagRecord.append(veh, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16000\2984826684.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

LaneID:28925528_0;Time:7206.0;
LaneID:28925528_0;Time:7207.0;
LaneID:28925528_0;Time:7208.0;
LaneID:28925528_0;Time:7209.0;
